# Introduction

In [14]:
# imports
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns  # for nicer plots
sns.set(style="darkgrid")  # default style
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

# baseline classifier
from sklearn.dummy import DummyRegressor

# random forest
from sklearn.ensemble import RandomForestRegressor

import tensorflow as tf

gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

GPU is available


In [15]:
# load metadata
train_images = os.listdir('petfinder-pawpularity-score/train')
train_df = pd.read_csv('petfinder-pawpularity-score/train.csv')
train_df = train_df.set_index('Id')

In [16]:
train_df

,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity
Id,,,,,,,,,,,,,
0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63
0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42
0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28
0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15
001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffbfa0383c34dc513c95560d6e1fdb57,0,0,0,1,0,0,0,0,0,0,0,1,15
ffcc8532d76436fc79e50eb2e5238e45,0,1,1,1,0,0,0,0,0,0,0,0,70
ffdf2e8673a1da6fb80342fa3b119a20,0,1,1,1,0,0,0,0,1,1,0,0,20


# Base Model: Predict Mean Pawpularity Score

In [17]:
y = train_df['Pawpularity'].to_numpy()
X = train_df.drop(columns=['Pawpularity']).to_numpy()

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=137)

dummy_model = DummyRegressor(strategy='mean')
scores = cross_val_score(dummy_model, X, y, cv=10, scoring='neg_root_mean_squared_error')
print("%0.2f RMSE with a standard deviation of %0.2f" % (-scores.mean(), scores.std()))

20.58 RMSE with a standard deviation of 0.56


In [18]:
scores

array([-20.3394095 , -21.05160374, -21.63166186, -19.96940898,
       -20.47178551, -21.34891135, -20.43489616, -19.79986834,
       -20.51705887, -20.28407691])

# EfficientNetB0

In [6]:
tf.keras.backend.clear_session()

In [7]:
train_images = os.listdir('petfinder-pawpularity-score/train')
train_df = pd.read_csv('petfinder-pawpularity-score/train.csv')
train_df = train_df.set_index('Id')

file_names = []
labels = []

for index in train_df.index:
    file_name = 'petfinder-pawpularity-score/train/' + index + '.jpg'
    label = train_df.loc[index]['Pawpularity']
    file_names.append(file_name)
    labels.append(label/100.01)
    #labels.append(label*1.0)

train_files, val_files, labels_train, labels_val = train_test_split(file_names, labels, test_size=0.2, random_state=137)

def process_file_names_train(file_name, label):
    image_string = tf.io.read_file(file_name)
    img = tf.io.decode_jpeg(image_string, channels=3)
    img = tf.cast(img, tf.float32)
    img = tf.image.resize(img, [224,224])
    img = tf.keras.applications.efficientnet.preprocess_input(img) 
    # drop this for val data
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_saturation(img, 0.95, 1.05)
    img = tf.image.random_contrast(img, 0.95, 1.05)
    

    #img = tf.image.random_hue(img, 0.05)
    #img = tf.image.random_saturation(img, 0.7, 1.3)
    #img = tf.image.random_contrast(img, 0.8, 1.2)
    #img = tf.image.random_brightness(img, 0.15)
    return img, label

def process_file_names_val(file_name, label):
    image_string = tf.io.read_file(file_name)
    img = tf.io.decode_jpeg(image_string, channels=3)
    img = tf.cast(img, tf.float32)
    img = tf.image.resize(img, [224,224])
    img = tf.keras.applications.efficientnet.preprocess_input(img) 
    return img, label

train_dataset = tf.data.Dataset.from_tensor_slices((train_files, labels_train))
train_dataset = train_dataset.map(process_file_names_train)

val_dataset = tf.data.Dataset.from_tensor_slices((val_files, labels_val))
val_dataset = val_dataset.map(process_file_names_val)

def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(32)
    return ds

train_dataset = configure_for_performance(train_dataset)

val_dataset = configure_for_performance(val_dataset)

2024-04-13 17:24:09.035728: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-04-13 17:24:09.035754: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-04-13 17:24:09.035759: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-04-13 17:24:09.035786: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-13 17:24:09.035800: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
base_model = tf.keras.applications.efficientnet.EfficientNetB0(include_top=False, pooling='avg')
base_model.trainable = False

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(224, 224, 3)),
    base_model,
    tf.keras.layers.Dense(units = 16, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    #tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units = 1, activation="sigmoid")
])

early_stopping = tf.keras.callbacks.EarlyStopping(patience = 5,restore_best_weights=True)

    
lr_schedule = tf.keras.optimizers.schedules.CosineDecay(1e-4, 10000, warmup_target=1e-3, warmup_steps=1000, alpha=1e-4)


model.compile(loss="binary_crossentropy", 
              optimizer = tf.keras.optimizers.legacy.Adam(learning_rate = lr_schedule), 
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

predictor = model.fit(train_dataset,
                      epochs=50, 
                      validation_data = val_dataset,
                      callbacks=[early_stopping])

Epoch 1/50


2024-04-13 17:24:18.948258: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


248/248 [==============================] - 82s 287ms/step - loss: 0.7385 - root_mean_squared_error: 0.2692 - val_loss: 0.6761 - val_root_mean_squared_error: 0.2235
Epoch 2/50
248/248 [==============================] - 75s 300ms/step - loss: 0.6666 - root_mean_squared_error: 0.2063 - val_loss: 0.6619 - val_root_mean_squared_error: 0.2089
Epoch 3/50
248/248 [==============================] - 74s 296ms/step - loss: 0.6536 - root_mean_squared_error: 0.1908 - val_loss: 0.6577 - val_root_mean_squared_error: 0.2029
Epoch 4/50
248/248 [==============================] - 72s 289ms/step - loss: 0.6491 - root_mean_squared_error: 0.1855 - val_loss: 0.6516 - val_root_mean_squared_error: 0.1967
Epoch 5/50
248/248 [==============================] - 73s 292ms/step - loss: 0.6466 - root_mean_squared_error: 0.1823 - val_loss: 0.6501 - val_root_mean_squared_error: 0.1952
Epoch 6/50
248/248 [==============================] - 73s 292ms/step - loss: 0.6455 - root_mean_squared_error: 0.1810 - val_loss: 0.6494

# EfficientNetB2

In [ ]:
tf.keras.backend.clear_session()


train_images = os.listdir('petfinder-pawpularity-score/train')
train_df = pd.read_csv('petfinder-pawpularity-score/train.csv')
train_df = train_df.set_index('Id')

file_names = []
labels = []

for index in train_df.index:
    file_name = 'petfinder-pawpularity-score/train/' + index + '.jpg'
    label = train_df.loc[index]['Pawpularity']
    file_names.append(file_name)
    labels.append(label/100.01)
    #labels.append(label*1.0)

train_files, val_files, labels_train, labels_val = train_test_split(file_names, labels, test_size=0.2, random_state=137)

def process_file_names_train(file_name, label):
    image_string = tf.io.read_file(file_name)
    img = tf.io.decode_jpeg(image_string, channels=3)
    img = tf.cast(img, tf.float32)
    img = tf.image.resize(img, [224,224])
    img = tf.keras.applications.efficientnet.preprocess_input(img) 
    # drop this for val data
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_saturation(img, 0.95, 1.05)
    img = tf.image.random_contrast(img, 0.95, 1.05)
    

    #img = tf.image.random_hue(img, 0.05)
    #img = tf.image.random_saturation(img, 0.7, 1.3)
    #img = tf.image.random_contrast(img, 0.8, 1.2)
    #img = tf.image.random_brightness(img, 0.15)
    return img, label

def process_file_names_val(file_name, label):
    image_string = tf.io.read_file(file_name)
    img = tf.io.decode_jpeg(image_string, channels=3)
    img = tf.cast(img, tf.float32)
    img = tf.image.resize(img, [224,224])
    img = tf.keras.applications.efficientnet.preprocess_input(img) 
    return img, label

train_dataset = tf.data.Dataset.from_tensor_slices((train_files, labels_train))
train_dataset = train_dataset.map(process_file_names_train)

val_dataset = tf.data.Dataset.from_tensor_slices((val_files, labels_val))
val_dataset = val_dataset.map(process_file_names_val)

def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(32)
    return ds

train_dataset = configure_for_performance(train_dataset)

val_dataset = configure_for_performance(val_dataset)

In [9]:
base_model = tf.keras.applications.efficientnet.EfficientNetB2(include_top=False, pooling='avg')
base_model.trainable = False

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(224, 224, 3)),
    base_model,
    tf.keras.layers.Dense(units = 16, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    #tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units = 1, activation="sigmoid")
])

early_stopping = tf.keras.callbacks.EarlyStopping(patience = 5,restore_best_weights=True)

    
lr_schedule = tf.keras.optimizers.schedules.CosineDecay(1e-4, 10000, warmup_target=1e-3, warmup_steps=1000, alpha=1e-4)


model.compile(loss="binary_crossentropy", 
              optimizer = tf.keras.optimizers.legacy.Adam(learning_rate = lr_schedule), 
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

predictor = model.fit(train_dataset,
                      epochs=50, 
                      validation_data = val_dataset,
                      callbacks=[early_stopping])

Epoch 1/50
248/248 [==============================] - 121s 459ms/step - loss: 0.7347 - root_mean_squared_error: 0.2664 - val_loss: 0.6940 - val_root_mean_squared_error: 0.2422
Epoch 2/50
248/248 [==============================] - 106s 428ms/step - loss: 0.6672 - root_mean_squared_error: 0.2071 - val_loss: 0.6608 - val_root_mean_squared_error: 0.2076
Epoch 3/50
248/248 [==============================] - 104s 417ms/step - loss: 0.6533 - root_mean_squared_error: 0.1905 - val_loss: 0.6549 - val_root_mean_squared_error: 0.2002
Epoch 4/50
248/248 [==============================] - 105s 421ms/step - loss: 0.6491 - root_mean_squared_error: 0.1853 - val_loss: 0.6524 - val_root_mean_squared_error: 0.1978
Epoch 5/50
248/248 [==============================] - 104s 417ms/step - loss: 0.6472 - root_mean_squared_error: 0.1831 - val_loss: 0.6503 - val_root_mean_squared_error: 0.1954
Epoch 6/50
248/248 [==============================] - 104s 417ms/step - loss: 0.6457 - root_mean_squared_error: 0.1812 -

# EfficientNetB4

In [10]:
tf.keras.backend.clear_session()


train_images = os.listdir('petfinder-pawpularity-score/train')
train_df = pd.read_csv('petfinder-pawpularity-score/train.csv')
train_df = train_df.set_index('Id')

file_names = []
labels = []

for index in train_df.index:
    file_name = 'petfinder-pawpularity-score/train/' + index + '.jpg'
    label = train_df.loc[index]['Pawpularity']
    file_names.append(file_name)
    labels.append(label/100.01)
    #labels.append(label*1.0)

train_files, val_files, labels_train, labels_val = train_test_split(file_names, labels, test_size=0.2, random_state=137)

def process_file_names_train(file_name, label):
    image_string = tf.io.read_file(file_name)
    img = tf.io.decode_jpeg(image_string, channels=3)
    img = tf.cast(img, tf.float32)
    img = tf.image.resize(img, [224,224])
    img = tf.keras.applications.efficientnet.preprocess_input(img) 
    # drop this for val data
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_saturation(img, 0.95, 1.05)
    img = tf.image.random_contrast(img, 0.95, 1.05)
    

    #img = tf.image.random_hue(img, 0.05)
    #img = tf.image.random_saturation(img, 0.7, 1.3)
    #img = tf.image.random_contrast(img, 0.8, 1.2)
    #img = tf.image.random_brightness(img, 0.15)
    return img, label

def process_file_names_val(file_name, label):
    image_string = tf.io.read_file(file_name)
    img = tf.io.decode_jpeg(image_string, channels=3)
    img = tf.cast(img, tf.float32)
    img = tf.image.resize(img, [224,224])
    img = tf.keras.applications.efficientnet.preprocess_input(img) 
    return img, label

train_dataset = tf.data.Dataset.from_tensor_slices((train_files, labels_train))
train_dataset = train_dataset.map(process_file_names_train)

val_dataset = tf.data.Dataset.from_tensor_slices((val_files, labels_val))
val_dataset = val_dataset.map(process_file_names_val)

def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(32)
    return ds

train_dataset = configure_for_performance(train_dataset)

val_dataset = configure_for_performance(val_dataset)

In [11]:
base_model = tf.keras.applications.efficientnet.EfficientNetB4(include_top=False, pooling='avg')
base_model.trainable = False

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(224, 224, 3)),
    base_model,
    tf.keras.layers.Dense(units = 16, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    #tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units = 1, activation="sigmoid")
])

early_stopping = tf.keras.callbacks.EarlyStopping(patience = 5,restore_best_weights=True)

    
lr_schedule = tf.keras.optimizers.schedules.CosineDecay(1e-4, 10000, warmup_target=1e-3, warmup_steps=1000, alpha=1e-4)


model.compile(loss="binary_crossentropy", 
              optimizer = tf.keras.optimizers.legacy.Adam(learning_rate = lr_schedule), 
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

predictor = model.fit(train_dataset,
                      epochs=50, 
                      validation_data = val_dataset,
                      callbacks=[early_stopping])

71686520/71686520 [==============================] - 3s 0us/step
Epoch 1/50
248/248 [==============================] - 196s 722ms/step - loss: 0.7391 - root_mean_squared_error: 0.2693 - val_loss: 0.6701 - val_root_mean_squared_error: 0.2176
Epoch 2/50
248/248 [==============================] - 170s 683ms/step - loss: 0.6659 - root_mean_squared_error: 0.2055 - val_loss: 0.6569 - val_root_mean_squared_error: 0.2028
Epoch 3/50
248/248 [==============================] - 167s 673ms/step - loss: 0.6542 - root_mean_squared_error: 0.1916 - val_loss: 0.6558 - val_root_mean_squared_error: 0.2011
Epoch 4/50
248/248 [==============================] - 168s 674ms/step - loss: 0.6497 - root_mean_squared_error: 0.1861 - val_loss: 0.6526 - val_root_mean_squared_error: 0.1981
Epoch 5/50
248/248 [==============================] - 170s 685ms/step - loss: 0.6478 - root_mean_squared_error: 0.1838 - val_loss: 0.6517 - val_root_mean_squared_error: 0.1969
Epoch 6/50
248/248 [==============================] - 1

KeyboardInterrupt: 

# EfficientNetB0 More Complicated Head

In [6]:
tf.keras.backend.clear_session()


train_images = os.listdir('petfinder-pawpularity-score/train')
train_df = pd.read_csv('petfinder-pawpularity-score/train.csv')
train_df = train_df.set_index('Id')

file_names = []
labels = []

for index in train_df.index:
    file_name = 'petfinder-pawpularity-score/train/' + index + '.jpg'
    label = train_df.loc[index]['Pawpularity']
    file_names.append(file_name)
    labels.append(label/100.01)
    #labels.append(label*1.0)

train_files, val_files, labels_train, labels_val = train_test_split(file_names, labels, test_size=0.2, random_state=137)

def process_file_names_train(file_name, label):
    image_string = tf.io.read_file(file_name)
    img = tf.io.decode_jpeg(image_string, channels=3)
    img = tf.cast(img, tf.float32)
    img = tf.image.resize(img, [224,224])
    img = tf.keras.applications.efficientnet.preprocess_input(img) 
    # drop this for val data
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_saturation(img, 0.95, 1.05)
    img = tf.image.random_contrast(img, 0.95, 1.05)
    

    #img = tf.image.random_hue(img, 0.05)
    #img = tf.image.random_saturation(img, 0.7, 1.3)
    #img = tf.image.random_contrast(img, 0.8, 1.2)
    #img = tf.image.random_brightness(img, 0.15)
    return img, label

def process_file_names_val(file_name, label):
    image_string = tf.io.read_file(file_name)
    img = tf.io.decode_jpeg(image_string, channels=3)
    img = tf.cast(img, tf.float32)
    img = tf.image.resize(img, [224,224])
    img = tf.keras.applications.efficientnet.preprocess_input(img) 
    return img, label

train_dataset = tf.data.Dataset.from_tensor_slices((train_files, labels_train))
train_dataset = train_dataset.map(process_file_names_train)

val_dataset = tf.data.Dataset.from_tensor_slices((val_files, labels_val))
val_dataset = val_dataset.map(process_file_names_val)

def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(128)
    return ds

train_dataset = configure_for_performance(train_dataset)

val_dataset = configure_for_performance(val_dataset)

2024-04-13 19:58:01.223909: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-04-13 19:58:01.223934: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-04-13 19:58:01.223939: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-04-13 19:58:01.223968: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-13 19:58:01.223981: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
base_model = tf.keras.applications.efficientnet.EfficientNetB0(include_top=False, pooling='avg')
base_model.trainable = False

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(224, 224, 3)),
    base_model,
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units = 32, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units = 16, activation="relu"),
    tf.keras.layers.Dense(units = 1, activation="sigmoid")
])

early_stopping = tf.keras.callbacks.EarlyStopping(patience = 5,restore_best_weights=True)

    
lr_schedule = tf.keras.optimizers.schedules.CosineDecay(1e-4, 2000, warmup_target=1e-3, warmup_steps=200, alpha=1e-4)


model.compile(loss="binary_crossentropy", 
              optimizer = tf.keras.optimizers.legacy.Adam(learning_rate = lr_schedule), 
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

predictor = model.fit(train_dataset,
                      epochs=50, 
                      validation_data = val_dataset,
                      callbacks=[early_stopping])

Epoch 1/50


2024-04-13 19:58:07.137149: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


62/62 [==============================] - 92s 1s/step - loss: 0.7774 - root_mean_squared_error: 0.2904 - val_loss: 0.6655 - val_root_mean_squared_error: 0.2129
Epoch 2/50
62/62 [==============================] - 75s 1s/step - loss: 0.7089 - root_mean_squared_error: 0.2414 - val_loss: 0.6574 - val_root_mean_squared_error: 0.2036
Epoch 3/50
62/62 [==============================] - 74s 1s/step - loss: 0.6879 - root_mean_squared_error: 0.2251 - val_loss: 0.6539 - val_root_mean_squared_error: 0.1997
Epoch 4/50
62/62 [==============================] - 76s 1s/step - loss: 0.6696 - root_mean_squared_error: 0.2084 - val_loss: 0.6518 - val_root_mean_squared_error: 0.1975
Epoch 5/50
62/62 [==============================] - 73s 1s/step - loss: 0.6606 - root_mean_squared_error: 0.1990 - val_loss: 0.6517 - val_root_mean_squared_error: 0.1974
Epoch 6/50
62/62 [==============================] - 79s 1s/step - loss: 0.6557 - root_mean_squared_error: 0.1935 - val_loss: 0.6514 - val_root_mean_squared_error

In [8]:
print(np.mean(labels_train))
print(np.mean(labels_val))

0.3800490174213755
0.38156547431489835


In [11]:
np.sqrt(np.mean(np.square(labels_val - np.mean(labels_train))))

0.2127701997465911

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional  (None, 1280)              4049571   
 )                                                               
                                                                 
 dense (Dense)               (None, 32)                40992     
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 4091108 (15.61 MB)
Trainable params: 41537 (162.25 KB)
Non-trainable params: 4049571 (15.45 MB)
_________________________________________________________________


# trying out other approach in Kaggle

In [19]:
tf.keras.backend.clear_session()


train_images = os.listdir('petfinder-pawpularity-score/train')
train_df = pd.read_csv('petfinder-pawpularity-score/train.csv')
train_df = train_df.set_index('Id')

file_names = []
labels = []

for index in train_df.index:
    file_name = 'petfinder-pawpularity-score/train/' + index + '.jpg'
    label = train_df.loc[index]['Pawpularity']
    file_names.append(file_name)
    labels.append(label/100.01)
    #labels.append(label*1.0)

train_files, val_files, labels_train, labels_val = train_test_split(file_names, labels, test_size=0.2, random_state=137)

def process_file_names_train(file_name, label):
    image_string = tf.io.read_file(file_name)
    img = tf.io.decode_jpeg(image_string, channels=3)
    img = tf.cast(img, tf.float32)
    img = tf.image.resize(img, [224,224])
    #img = tf.keras.applications.efficientnet.preprocess_input(img) 
    

    #img = tf.image.random_hue(img, 0.05)
    #img = tf.image.random_saturation(img, 0.7, 1.3)
    #img = tf.image.random_contrast(img, 0.8, 1.2)
    #img = tf.image.random_brightness(img, 0.15)
    return img, label

def process_file_names_val(file_name, label):
    image_string = tf.io.read_file(file_name)
    img = tf.io.decode_jpeg(image_string, channels=3)
    img = tf.cast(img, tf.float32)
    img = tf.image.resize(img, [224,224])
    #img = tf.keras.applications.efficientnet.preprocess_input(img) 
    return img, label

train_dataset = tf.data.Dataset.from_tensor_slices((train_files, labels_train))
train_dataset = train_dataset.map(process_file_names_train)

val_dataset = tf.data.Dataset.from_tensor_slices((val_files, labels_val))
val_dataset = val_dataset.map(process_file_names_val)

def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(512)
    return ds

train_dataset = configure_for_performance(train_dataset)

val_dataset = configure_for_performance(val_dataset)

In [20]:
base_model = tf.keras.applications.efficientnet.EfficientNetB2(include_top=False, pooling='avg')
base_model.trainable = False

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(224, 224, 3)),
    base_model,
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units = 128, activation="relu"),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(units = 32, activation="relu"),
    tf.keras.layers.Dense(units = 1, activation="sigmoid")
])

early_stopping = tf.keras.callbacks.EarlyStopping(patience = 3,restore_best_weights=True)

    
lr_schedule = tf.keras.optimizers.schedules.CosineDecay(1e-4, 2000, warmup_target=1e-3, warmup_steps=200, alpha=1e-4)


model.compile(loss="mse", 
              optimizer = tf.keras.optimizers.legacy.Adam(learning_rate = 1e-4), 
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

predictor = model.fit(train_dataset,
                      epochs=20, 
                      validation_data = val_dataset,
                      callbacks=[early_stopping])

Epoch 1/20
16/16 [==============================] - 129s 8s/step - loss: 0.0872 - root_mean_squared_error: 0.2954 - val_loss: 0.0632 - val_root_mean_squared_error: 0.2514
Epoch 2/20
16/16 [==============================] - 109s 7s/step - loss: 0.0709 - root_mean_squared_error: 0.2663 - val_loss: 0.0491 - val_root_mean_squared_error: 0.2215
Epoch 3/20
16/16 [==============================] - 111s 7s/step - loss: 0.0666 - root_mean_squared_error: 0.2580 - val_loss: 0.0482 - val_root_mean_squared_error: 0.2196
Epoch 4/20
16/16 [==============================] - 109s 7s/step - loss: 0.0628 - root_mean_squared_error: 0.2506 - val_loss: 0.0478 - val_root_mean_squared_error: 0.2187
Epoch 5/20
16/16 [==============================] - 110s 7s/step - loss: 0.0611 - root_mean_squared_error: 0.2473 - val_loss: 0.0456 - val_root_mean_squared_error: 0.2136
Epoch 6/20
16/16 [==============================] - 110s 7s/step - loss: 0.0583 - root_mean_squared_error: 0.2414 - val_loss: 0.0446 - val_root_m

In [21]:
predictor = model.fit(train_dataset,
                      epochs=20, 
                      validation_data = val_dataset,
                      callbacks=[early_stopping])

Epoch 1/20
16/16 [==============================] - 113s 7s/step - loss: 0.0458 - root_mean_squared_error: 0.2140 - val_loss: 0.0392 - val_root_mean_squared_error: 0.1981
Epoch 2/20
16/16 [==============================] - 109s 7s/step - loss: 0.0446 - root_mean_squared_error: 0.2111 - val_loss: 0.0390 - val_root_mean_squared_error: 0.1976
Epoch 3/20
16/16 [==============================] - 108s 7s/step - loss: 0.0455 - root_mean_squared_error: 0.2132 - val_loss: 0.0389 - val_root_mean_squared_error: 0.1973
Epoch 4/20
16/16 [==============================] - 110s 7s/step - loss: 0.0445 - root_mean_squared_error: 0.2111 - val_loss: 0.0388 - val_root_mean_squared_error: 0.1971
Epoch 5/20
16/16 [==============================] - 109s 7s/step - loss: 0.0427 - root_mean_squared_error: 0.2065 - val_loss: 0.0387 - val_root_mean_squared_error: 0.1968
Epoch 6/20
16/16 [==============================] - 108s 7s/step - loss: 0.0432 - root_mean_squared_error: 0.2078 - val_loss: 0.0386 - val_root_m